In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
data = pd.read_stata("KalmanFile1.dta")

In [3]:
data1 = data[data['crsp_cl_grp'] == 2017461.0].reset_index(drop = True)

In [4]:
data1['year'] = data1['year'].astype(int) 

In [5]:
data1.columns = ['$Fund$','year','quarter','$q_t$','$f_{t}$','$r_t$','$\\sigma_t$','$\\sigma_t^*$','$t$']

In [6]:
data1[['$r_{t+1}$','$f_{t+1}$','$\\sigma_{t+1}$','$\\sigma_{t+1}^*$','$q_{t+1}$']] = data1[['$r_t$','$f_{t}$','$\\sigma_t$','$\\sigma_t^*$','$q_t$']].shift(periods =-1)

In [7]:
data1['$\\Delta q_{t+1}$'] = data1['$q_{t+1}$']-data1['$q_t$']
data1['$\\sigma_{t+1}^2$'] = data1['$\\sigma_{t+1}$']**2
data1['$\\sigma_t^2$'] = data1['$\\sigma_t$']**2
data1['$\\Delta\\sigma_{t+1}^2$'] = data1['$\\sigma_{t+1}^2$']-data1['$\\sigma_t^2$']
data1['$\\Delta CV_{t+1}$'] = data1['$\\Delta\\sigma_{t+1}^2$']/data1['$r_{t+1}$']

In [8]:
df = data1[['$Fund$', '$t$', '$r_{t+1}$','$\Delta q_{t+1}$', '$\sigma_{t+1}^2$','$\Delta CV_{t+1}$','$\\Delta\\sigma_{t+1}^2$']]

In [9]:
df = df.dropna().reset_index(drop=True)

In [10]:
df['$y_t$'] = df['$\Delta q_{t+1}$']/df['$r_{t+1}$']

In [11]:
df

,$Fund$,$t$,$r_{t+1}$,$\Delta q_{t+1}$,$\sigma_{t+1}^2$,$\Delta CV_{t+1}$,$\Delta\sigma_{t+1}^2$,$y_t$
0,2017461.0,1999-10-01,25.016842,42.200012,591.157654,13.500421,337.737885,1.686864
1,2017461.0,2000-01-01,-33.653526,-65.599976,841.011719,-7.424305,249.854065,1.949275
2,2017461.0,2000-04-01,18.666679,82.399963,231.188538,-32.669075,-609.823181,4.414281
3,2017461.0,2000-07-01,-21.791695,-22.699951,754.213684,-24.001123,523.025146,1.041679
4,2017461.0,2000-10-01,32.792274,91.299988,658.444397,-2.920483,-95.769287,2.784192
...,...,...,...,...,...,...,...,...
73,2017461.0,2018-01-01,5.292100,-10.199982,154.426804,-42.608372,-225.487778,-1.927398
74,2017461.0,2018-04-01,-12.255882,-11.200012,54.902573,8.120527,-99.524231,0.913848
75,2017461.0,2018-07-01,-18.435755,-95.399994,586.651611,-28.843355,531.749023,5.174727
76,2017461.0,2018-10-01,4.173483,41.799988,197.973831,-93.130318,-388.677795,10.015613


# Transition
## $$x_{t+1} = A x_t + B \epsilon_{t+1}$$ 

In [12]:
A = np.array([[0,0,0,0],[0,0,0,0],[0,1,1,0],[0,0,1,0]])

In [13]:
A

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 0]])

In [14]:
B = np.array([[0,1],[1,0],[0,0],[0,0]])

In [15]:
B

array([[0, 1],
       [1, 0],
       [0, 0],
       [0, 0]])

$\epsilon_{t+1} = White Noise$ ~$N(0,1)$

In [16]:
E = np.array([[np.random.normal(0,1),np.random.normal(0,1)]]).transpose()

In [17]:
E

array([[1.04577871],
       [0.19702137]])

# initial $x_t$

## $$x_t\begin{equation*} = \begin{pmatrix}\lambda_{\alpha t}\\\lambda_{\psi t}\\\psi_{t+1}\\\psi_{t}\end{pmatrix}= \begin{pmatrix}0\\0\\0\\0\end{pmatrix}\end{equation*} $$

In [18]:
xt1i = np.array([[0,0,0,0]]).transpose()

In [19]:
xt1i

array([[0],
       [0],
       [0],
       [0]])

## $$x_{t+1} = A x_t + B \epsilon_{t+1}$$ 

In [20]:
xt1 = A.dot(xt1i) + B.dot(E)

In [21]:
xt1

array([[0.19702137],
       [1.04577871],
       [0.        ],
       [0.        ]])

## Measure

## $$Y_{t+1} = C_{t+1} x_{t+1} + D \epsilon_{t+1}$$

$$C = [1~~\sigma_{t+1}^2~~ 0~~ \Delta\sigma_{t+1}^2]$$

In [22]:
C = np.array([[1,df['$\sigma_{t+1}^2$'][0],0,df['$\\Delta\\sigma_{t+1}^2$'][0]]])

In [23]:
C

array([[  1.        , 591.15765381,   0.        , 337.73788452]])

In [24]:
D = np.array([[0,0]])

In [25]:
D

array([[0, 0]])

## $$\hat{Y_{t+1}} = C_{t+1} x_{t+1}$$

In [26]:
y_hat = C.dot(xt1)

In [27]:
y_hat

array([[618.41710897]])

## $$ error = Y_{t+1} - \hat{Y_{t+1}}$$

In [28]:
error = df['$y_t$'][0] - y_hat

In [29]:
error

array([[-616.73024484]])

In [30]:
#error in the estimate
Q = np.eye(4)

In [31]:
## initial Process covariance matrix
P_i = np.zeros_like(np.arange(16).reshape(4, 4) )

In [32]:
# predicted process covariance matrix
P_t = P_i + Q 

In [33]:
P_t

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## Kalman Gain

## $$K = \frac{P_tC'}{CP_tC'+R}$$

$$R = error~in ~the~ observations~(y_t)$$

In [34]:
## Kalman Gain
Kalman = P_t.dot(C.transpose())/(C.dot(P_t).dot(C.transpose())+np.var(df['$y_t$']))
## Updated estimates
xt1_hat = xt1 + Kalman*error

## $$\hat{x_{t+1}} = x_{t+1} + K * error$$

In [35]:
xt1_hat

array([[ 0.19569995],
       [ 0.26460731],
       [ 0.        ],
       [-0.4462958 ]])

## $$\hat{x_t}\begin{equation*} = \begin{pmatrix}\lambda_{\alpha t}\\\lambda_{\psi t}\\\psi_{t+1}\\\psi_{t}\end{pmatrix}= \begin{pmatrix}-0.9117967\\0.10754023\\0\\-0.17564115\end{pmatrix}\end{equation*} $$

In [36]:
#updated Y_t
C.dot(xt1_hat)

array([[5.88933503]])